This notebook contains an example SMT-encoding for the example `simple_proc.smv` for Bounded Model Checking. We follow chapter 8 of the [lecture notes of Fabio Somenzi](https://tuwel.tuwien.ac.at/mod/resource/view.php?id=2311706). Let us first import z3, this is the SMT solver we will use to encode the problem and solve it.

In [1]:
from z3 import *
set_param(proof=True)

As opposed to SAT-Solvers, z3 can handle more complex theories, e.g. encode enumeration types natively.

In [2]:
State, (ready, busy) = EnumSort('State', ('ready', 'busy'))

We will now encode the specification from `examples/short.smv` into SMT. Specifically, we want to check whether there is a $(k,l)$-loop (see [page 370](https://tuwel.tuwien.ac.at/mod/resource/view.php?id=2311706)) which violates one of the LTL properties.

In [3]:
# The states of the loop are simply the integers from 0 to k.  For convenience,
# we define a function that gives us the next state.
def succ(k, l, i):
    return (i + 1) if i < k else l

# First we must declare all variables that will be used in the model. Each state
# of the (k,l)-loop will have a request variable and a state variable.
def variables(k):
    return {f'request{i}' : Bool(f'request{i}') for i in range(k + 1)} | \
        {f'state{i}' : Const(f'state{i}', State) for i in range(k + 1)}

# We now define the constraints on the (k,l)-loop that come from the transition
# relation.
def init_constraint(vars):
    # The initial state of the loop is ready.
    return vars['state0'] == ready

def trans_constraint(k, l, vars):
    # If state i is ready and request i is true, then state i+1 is busy.
    return And(*[
        Implies(
            And(vars[f'state{i}'] == ready, vars[f'request{i}']),
            vars[f'state{succ(k, l, i)}'] == busy
        ) for i in range(k + 1)
    ])

Let us check if there is a $(0,0)$-loop modelling this transition system, without any additional specifications:

In [4]:
s = Solver()
vars = variables(0)
s.add(init_constraint(vars))
s.add(trans_constraint(0, 0, vars))
print(s.check())

sat


The solver returned `sat`, meaning it has found a model, let's check it out:

In [5]:
print(s.model())

[request0 = False, state0 = ready]


Let us now encode the LTL properties into SMT.

In [6]:
# We define a helper function that, given k, l and a state i, returns a formula
# that encodes if F state = busy holds in that state.
def F_state_busy(k, l, i, vars):
    return Or(*[
        vars[f'state{j}'] == busy for j in (
            range(i, k + 1) if i < l else range(l, k + 1)
        )
    ])

def LTLSPEC1(k, l, vars):
    return And(*[
        Implies(
            vars[f'request{i}'],
            F_state_busy(k, l, i, vars)
        ) for i in range(k + 1)
    ])

def LTLSPEC2(k, l, vars):
    return And(*[
        Implies(
            vars[f'request{i}'],
            F_state_busy(k, l, succ(k, l, i), vars)
        ) for i in range(k + 1)
    ])

We define now two methods which check for each LTL property if there is a $(k,l)$-loop violating it.

In [7]:
def encoding1(k, l): 
    s = Solver()
    vars = variables(k)
    s.add(init_constraint(vars))
    s.add(trans_constraint(k, l, vars))
    s.add(Not(LTLSPEC1(k, l, vars)))
    return s

def encoding2(k, l):
    s = Solver()
    vars = variables(k)
    s.add(init_constraint(vars))
    s.add(trans_constraint(k, l, vars))
    s.add(Not(LTLSPEC2(k, l, vars)))
    return s

We see that there is no $(k,l)$-loop violating property 1 for $k \leq 10$.

In [8]:
for k, l in [(k, l) for k in range(1, 10) for l in range(1, k + 1)]:
    s = encoding1(k, l)
    if s.check() == sat:
        print(f'LTLSPEC1({k}, {l}) is not satisfied')
        print(s.model())
        break
else:
    print('LTLSPEC1 is satisfied for all k, l <= 10')

LTLSPEC1 is satisfied for all k, l <= 10


On the other hand, there is a $(2,2)$-loop violating property 2.

In [9]:
for k, l in [(k, l) for k in range(1, 10) for l in range(1, k + 1)]:
    s = encoding2(k, l)
    if s.check() == sat:
        print(f'LTLSPEC2({k}, {l}) is not satisfied')
        print(s.model())
        break
else:
    print('LTLSPEC2 is satisfied for all k, l <= 10')

LTLSPEC2(2, 2) is not satisfied
[request1 = True,
 request2 = False,
 state2 = ready,
 state1 = busy,
 state0 = ready]
